In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

In [2]:
train_data = data.filter(pl.col("Group") != "5")
test_data = data.filter(pl.col("Group") == "5")

In [3]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [4]:
data.select('Region').unique()

Region
str
"""Haute-Normandie"""
"""Limousin"""
"""Franche-Comte"""
"""Rhone-Alpes"""
"""Champagne-Ardenne"""
…
"""Poitou-Charentes"""
"""Bourgogne"""
"""Languedoc-Roussillon"""


In [5]:
# fits model
model = rs.glm(
    formula="ClaimCount ~ TE(Region) + TE(Area) + TE(VehBrand) + VehAge + BonusMalus + DrivAge",
    data=train_data,
    family="negbinomial",
    offset="Exposure"
).fit()

model2 = rs.glm(
    formula="ClaimCount ~ TE(Region) + TE(Area) + VehAge + BonusMalus + DrivAge",
    data=train_data,
    family="negbinomial",
    offset="Exposure"
).fit()

In [6]:
print(model.summary())

                                 GLM Results                                  

Family:              NegativeBinomial(theta=1.8927) No. Observations:        542055
Link Function:       (default)       Df Residuals:            542048
Method:              IRLS + Ridge    Df Model:                     6
Scale:               0.2905          Alpha (λ):               0.0000
L1 Ratio:            0.00            Iterations:                   6
Non-zero coefs:      6              

Log-Likelihood:         -112675.1525 Deviance:                157458.8692
AIC:                     225364.3049 Null Deviance:            58099.8532
BIC:                     225442.7268 Pearson chi2:             1312678.64
Converged:           True           

------------------------------------------------------------------------------
Variable               Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept          

In [7]:
#writes analysis/diagnostics
model.diagnostics(
    train_data=train_data,
    test_data = test_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ TE(Region) + TE(Area) + TE(VehBrand) + VehAge + BonusMalus + DrivAge', 'family': 'NegativeBinomial(theta=1.8927)', 'link': 'log', 'n_obs': 542055, 'n_params': 7, 'df_resid': 542048, 'converged': True, 'iterations': 6}, fit_statistics={'deviance': 157458.869177, 'null_deviance': 58099.85321030604, 'deviance_explained': -1.7101422891214662, 'log_likelihood': -112675.15247416665, 'aic': 225364.3049483333, 'bic': 225442.72680759232, 'pearson_chi2': 1312678.6433971364, 'dispersion': 2.421701848170524}, loss_metrics={'family_deviance_loss': 0.29048548196738366, 'mse': 0.05422603089995575, 'mae': 0.09764440518340903, 'rmse': 0.23286483397017196}, calibration={'ae_ratio': 0.993, 'hl_pvalue': 0.0, 'problem_deciles': [{'decile': 0, 'ae': 2.33, 'n': 196839, 'ae_ci': [2.27, 2.39]}, {'decile': 1, 'ae': 1.2, 'n': 64192, 'ae_ci': [1.15, 1.24]}, {'decile': 3, 'ae': 0.86, 'n': 36902, 'ae_ci': [0.82, 0.89]}, {'decile': 4, 'ae': 0.81, 'n': 34403, '

In [8]:
#writes analysis/diagnostics
model2.diagnostics(
    train_data=train_data,
    test_data = test_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ TE(Region) + TE(Area) + VehAge + BonusMalus + DrivAge', 'family': 'NegativeBinomial(theta=1.9143)', 'link': 'log', 'n_obs': 542055, 'n_params': 6, 'df_resid': 542049, 'converged': True, 'iterations': 6}, fit_statistics={'deviance': 157676.6636363408, 'null_deviance': 58201.68653186428, 'deviance_explained': -1.7091425185766038, 'log_likelihood': -112722.04487160443, 'aic': 225456.08974320887, 'bic': 225523.3084797166, 'pearson_chi2': 1335061.476992911, 'dispersion': 2.462990388309749}, loss_metrics={'family_deviance_loss': 0.2908866976062958, 'mse': 0.05420662315383303, 'mae': 0.0975983502646381, 'rmse': 0.2328231585427726}, calibration={'ae_ratio': 0.993, 'hl_pvalue': 0.0, 'problem_deciles': [{'decile': 0, 'ae': 2.33, 'n': 196882, 'ae_ci': [2.27, 2.39]}, {'decile': 1, 'ae': 1.23, 'n': 64091, 'ae_ci': [1.18, 1.27]}, {'decile': 3, 'ae': 0.86, 'n': 37211, 'ae_ci': [0.82, 0.9]}, {'decile': 4, 'ae': 0.82, 'n': 34638, 'ae_ci': [0.79, 